import argparse

parser = argparse.ArgumentParser()
parser.add_argument('filesignature',type=int)
parser.add_argument('--batch_size', type=int, default=10)#100 ( 100, 16,32,64,128) priority 3
parser.add_argument('--seed',type=int,default=1337)#4(CH)9(JAP)17(ITAL)
parser.add_argument('--hidden_size', type=int, default=100)#priority 2
parser.add_argument('--layers', type=int, default=1) #priority : 1.9 (it should learn regardless , but the quality differ ) 
parser.add_argument('--batch_norm', action="store_true", default=False)#priority 5 , keep turned off
parser.add_argument('--no_batch_norm', action="store_false", dest='batch_norm')
parser.add_argument('--replay_size', type=int, default=100000)# try increasing later  , priority 3.1
parser.add_argument('--train_repeat', type=int, default=1)#(2^2) , priority 1
parser.add_argument('--gamma', type=float, default=0.99)# (calculated should be 0.99) (0.99)
parser.add_argument('--tau', type=float, default=0.001)# priority 0.9 (0.001 , 0.01 , 0.1) the one that work expeirment in the domain.
parser.add_argument('--totalsteps', type=int, default=1000000)# much more ( 1000 -> 10,000) (should be around 1 million steps)
parser.add_argument('--max_timesteps', type=int, default=1000)# 1000 
parser.add_argument('--activation', choices=['tanh', 'relu'], default='relu')# experiment ( relu , tanh) priority 0.7
parser.add_argument('--optimizer', choices=['adam', 'rmsprop'], default='adam')# priority 4.9
#parser.add_argument('--optimizer_lr', type=float, default=0.001)#could be used later priority 4.5
parser.add_argument('--exploration', type=float, default=0.1)# priority (0.8) it should decrease over time to reach 0.001 or even 0
parser.add_argument('--vanish',type=float,default=0.75)#Decide when the exploration should stop in percentage (75%)
parser.add_argument('--advantage', choices=['naive', 'max', 'avg'], default='avg')# priority 2 maybe done once and stike with one 
parser.add_argument('--rwrdschem',nargs='+',default=[-10,1000,-0.1],type=float) #(calculated should be (1000 reward , -0.1 punish per step)
parser.add_argument('--svision',type=int,default=180)
parser.add_argument('--details',type=str,default='')
parser.add_argument('--train_m',type=str,default='')
parser.add_argument('--target_m',type=str,default='')
args = parser.parse_args()

In [1]:
File_Signature=5000
batch_size=32
seed=1337
hidden_size=100
layers=1
batch_norm=False
replay_size=100000
train_repeat=1
gammma=0.99
tau=0.001
totalsteps=100
max_timesteps=1000
activation='tanh'
optimizer='adam'
#parser.add_argument('--optimizer_lr', type=float, default=0.001)#could be used later priority 4.5
exploration=0.1
vanish=0.75
advantage='avg'
rwrdschem=[-10,1000,-0.1]
svision=180
details='trying with tesnorflow'
train_m=''
target_m=''
import numpy as np
np.random.seed(seed)
import skvideo.io
from PD_Map import DPMP
from Settings import *
from World import *
from Agent import *
from Obstacles import *
from Foods import *
from time import time
from copy import deepcopy
from buffer import Buffer
import os
from keras.layers import Input,Dense,Lambda
from keras.models import Model,K

Using Theano backend.


In [2]:
def SetupEnvironment():
    Start = time()

    #Add Pictures
    Settings.SetBlockSize(20)
    Settings.AddImage('Wall','Pics/wall.jpg')
    Settings.AddImage('Food','Pics/food.jpg')

    #Specify World Size
    Settings.WorldSize=(11,11)

    #Create Probabilities
    obs = np.zeros(Settings.WorldSize)
    ragnt = np.zeros(Settings.WorldSize)
    gagnt = np.zeros(Settings.WorldSize)
    food = np.zeros(Settings.WorldSize)
    obs[3:8,5] = 1
    ragnt[:,0] =1
    gagnt[:,10]=1
    food[:,4:7]=1
    food[3:8,5] = 0

    #Add Probabilities to Settings
    Settings.AddProbabilityDistribution('Obs',obs)
    Settings.AddProbabilityDistribution('ragnt',ragnt)
    Settings.AddProbabilityDistribution('gagnt',gagnt)
    Settings.AddProbabilityDistribution('food',food)

    #Create World Elements
    obs = Obstacles('Wall',Shape=np.array([[1],[1],[1],[1]]),PdstName='Obs')
    food = Foods('Food',PdstName='food')

    ragnt = Agent(Fname='Pics/ragent.jpg',Power=3,VisionAngle=svision,Range=-1,PdstName='ragnt')
    gagnt = Agent(Fname='Pics/gagent.jpg',VisionAngle=180,Range=-1,Power=10,ControlRange=2,PdstName='gagnt')

    game =World(RewardsScheme=rwrdschem,StepsLimit=max_timesteps)
    #Adding Agents in Order of Following the action
    game.AddAgents([ragnt,gagnt])
    game.AddObstacles([obs])
    game.AddFoods([food])
    Start = time()-Start
    print ('Taken:',Start)
    return game

def createLayers(insize,naction):
    x = Input(shape=insize)#env.observation_space.shape)
    if batch_norm:
      h = BatchNormalization()(x)
    else:
      h = x
    for i in range(layers):
      h = Dense(hidden_size, activation=activation)(h)
      if batch_norm and i != layers - 1:
        h = BatchNormalization()(h)
    y = Dense(naction + 1)(h)
    if advantage == 'avg':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.mean(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'max':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.max(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'naive':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:], output_shape=(naction,))(y)
    else:
      assert False

    return x, z
TestingCounter=0
def TryModel(model,game):
    print('Testing Target Model')
    global AIAgent,File_Signature,TestingCounter,DAgent
    TestingCounter+=1
    writer = skvideo.io.FFmpegWriter("output/{}/VID/{}_Test.avi".format(File_Signature,TestingCounter))
    #writer2 = skvideo.io.FFmpegWriter("output/{}/VID/{}_TestAG.avi".format(File_Signature,TestingCounter))
    game.GenerateWorld()
    game.Step()
    img = game.BuildImage()
    Start = time()
    episode_reward=0
    observation = AIAgent.Flateoutput()

    writer.writeFrame(np.array(img*255,dtype=np.uint8))
    for t in range(max_timesteps):
        s =np.array([observation])
        q = model.predict(s, batch_size=1)
        action = np.argmax(q[0])
        AIAgent.NextAction = Settings.PossibleActions[action]
        DAgent.DetectAndAstar()
        game.Step()
        writer.writeFrame(np.array(game.BuildImage()*255,dtype=np.uint8))
        #writer2.writeFrame(np.array(game.AgentViewPoint(AIAgent.ID)*255,dtype=np.uint8))
        observation = AIAgent.Flateoutput()
        reward = AIAgent.CurrentReward
        done = game.Terminated[0]

        #observation, reward, done, info = env.step(action)
        episode_reward += reward

        #print "reward:", reward
        if done:
            break

    writer.close()
    #writer2.close()
    if t>=999:
        plt.imsave('output/{}/PNG/{}_Test.png'.format(File_Signature,TestingCounter),img)
    else:
        os.remove("output/{}/VID/{}_Test.avi".format(File_Signature,TestingCounter))
        #os.remove("output/{}/VID/{}_TestAG.avi".format(File_Signature,TestingCounter))

    Start = time()-Start
    print("Testing Episode {} finished after {} timesteps, episode reward {} Tooks {}s, Total Progress:{}".format(i_episode, t, episode_reward,Start,progress))

game = SetupEnvironment()
AIAgent = game.agents[1001]
DAgent = game.agents[1002]
'''
input size :
Worldsize*(Agents Count+3)+Agents Count *4
worldsize*(Agents count +3(food,observed,obstacles)) + Agents count *4 (orintation per agent)
'''
#ishape =(Settings.WorldSize[0]*Settings.WorldSize[1]*(len(game.agents)+3)+ len(game.agents)*4,)
ishape =(Settings.WorldSize[0]*Settings.WorldSize[1]*(2+3)+ 2*4,)
game.GenerateWorld()
game.Step()
naction =  Settings.PossibleActions.shape[0]

if train_m=='':
    print('train default')
    x, z = createLayers(ishape,naction)
    model = Model(input=x, output=z)
    model.summary()
    model.compile(optimizer='adam', loss='mse')
else:
    model = load_model('cur_mod/{}/model.h5'.format(train_m))
if target_m=='':
    print('test from scractch')
    x, z = createLayers(ishape,naction)
    target_model = Model(input=x, output=z)
    target_model.set_weights(model.get_weights())
else:
    target_model = load_model('cur_mod/{}/target_model.h5'.format(target_m))

mem = Buffer(replay_size,ishape,(1,))
#Exploration decrease amount:
EDA = exploration/(totalsteps*vanish)
#Framse Size
fs = (Settings.WorldSize[0]*Settings.BlockSize[0],Settings.WorldSize[1]*Settings.BlockSize[1])
total_reward = 0
#Create Folder to store the output
if not os.path.exists('output/{}'.format(File_Signature)):
        os.makedirs('output/{}'.format(File_Signature))
        os.makedirs('output/{}/PNG'.format(File_Signature))
        os.makedirs('output/{}/VID'.format(File_Signature))
        os.makedirs('output/{}/MOD'.format(File_Signature))
        
progress=0
i_episode=0
while progress<totalsteps:
    i_episode+=1
    game.GenerateWorld()

    Start = time()
    #First Step only do the calculation of the current observations for all agents
    game.Step()
    #Recording Video
    img =game.BuildImage()
    episode_reward=0
    observation = AIAgent.Flateoutput()

    for t in range(max_timesteps):
        exploration = exploration-EDA
        if np.random.random() < exploration:
          action =AIAgent.RandomAction()
        else:
          s =np.array([observation])
          q = model.predict(s, batch_size=1)
          action = np.argmax(q[0])
        prev_ob = observation
        AIAgent.NextAction = Settings.PossibleActions[action]
        DAgent.DetectAndAstar()
        game.Step()
        observation = AIAgent.Flateoutput()
        reward = AIAgent.CurrentReward
        done = game.Terminated[0]
        #observation, reward, done, info = env.step(action)
        episode_reward += reward
        #print "reward:", reward
        mem.add(prev_ob,np.array([action]),reward,observation,done)
        for k in range(train_repeat):
            prestates,actions,rewards,poststates,terminals = mem.sample(batch_size)

            qpre = model.predict(prestates)
            qpost = model.predict(poststates)
            for i in range(qpre.shape[0]):
                if terminals[i]:
                    qpre[i, actions[i]] = rewards[i]
                else:
                    try:
                        qpre[i, actions[i]] = rewards[i] + gammma * np.amax(qpost[i])
                    except Exception as ex:
                        print(ex)
                        print('qpre.shape:{},i:{},actions:{}'.format(qpre.shape,i,actions[i]))
            model.train_on_batch(prestates, qpre)
            weights = model.get_weights()
            target_weights = target_model.get_weights()
            for i in range(len(weights)):
                target_weights[i] = tau * weights[i] + (1 - tau) * target_weights[i]
            target_model.set_weights(target_weights)

        if done:
            break
    if t>=999:

        plt.imsave('output/{}/PNG/{}_train.png'.format(File_Signature,i_episode),img)
    aiprob =0# DPMP(wmap,agindx,findx,t+1)
    Start = time()-Start
    t = t+1
    progress+=t
    
    print("Episode {} finished after {} timesteps, episode reward {} Tooks {}s, Total Progress:{}".format(i_episode, t, episode_reward,Start,progress))
    total_reward += episode_reward
    if i_episode%10==0:
        TryModel(target_model,game)
        print("Average reward per episode {}".format(total_reward /i_episode))
model.save('output/{}/MOD/model.h5'.format(File_Signature))
target_model.save('output/{}/MOD/target_model.h5'.format(File_Signature))
TryModel(target_model,game)

Taken: 0.2096107006072998
train default
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 613)           0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           61400       input_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 6)             606         dense_1[0][0]                    
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 5)             0           dense_2[0][0]                    
Total params: 62,006
Trainable params: 62,006
Non-t

In [3]:
File_Signature=5000
batch_size=32
seed=1337
hidden_size=100
layers=1
batch_norm=False
replay_size=100000
train_repeat=1
gammma=0.99
tau=0.001
totalsteps=100
max_timesteps=1000
activation='tanh'
optimizer='adam'
#parser.add_argument('--optimizer_lr', type=float, default=0.001)#could be used later priority 4.5
exploration=0.1
vanish=0.75
advantage='avg'
rwrdschem=[-10,1000,-0.1]
svision=180
details='trying with tesnorflow'
train_m=''
target_m=''
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [36]:
?tf.split

In [59]:
def CreatetfNetwork(insize,naction):
    inputl = tf.placeholder(tf.float32,shape=(None,insize))
    print('input:',inputl.get_shape())
    tf_W = tf.Variable(tf.truncated_normal((insize,hidden_size)))
    tf_b = tf.Variable(tf.zeros(hidden_size))
    tf_hidlayer= tf.tanh(tf.matmul(inputl,tf_W))
    print('hidden-layer:',tf_hidlayer.get_shape())
    tf_Adv,tf_v = tf.split(tf_hidlayer,2,axis=1)
    print('advantage:{},Value:{}'.format(tf_Adv.get_shape(),tf_v.get_shape()))
    tf_adv_w = tf.Variable(tf.truncated_normal((hidden_size//2,naction)))
    tf_adv_b = tf.Variable(tf.zeros(naction))
    tf_v_w = tf.Variable(tf.truncated_normal((hidden_size//2,1)))
    tf_v_b = tf.Variable(tf.zeros(1))
    tf_AdvL = tf.matmul(tf_Adv,tf_adv_w)+ tf_adv_b
    tf_VL= tf.matmul(tf_v,tf_v_w)+ tf_v_b
    if advantage == 'avg':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.mean(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'max':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.max(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'naive':
        tf_final= tf
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:], output_shape=(naction,))(y)
    else:
      assert False

In [60]:
CreatetfNetwork(613,5)

input: (?, 613)
hidden-layer: (?, 100)
advantage:(?, 50),Value:(?, 50)


In [ ]:
def createLayers(insize,naction):
    x = Input(shape=insize)#env.observation_space.shape)
    if batch_norm:
      h = BatchNormalization()(x)
    else:
      h = x
    for i in range(layers):
      h = Dense(hidden_size, activation=activation)(h)
      if batch_norm and i != layers - 1:
        h = BatchNormalization()(h)
    y = Dense(naction + 1)(h)
    if advantage == 'avg':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.mean(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'max':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:] - K.max(a[:, 1:], keepdims=True), output_shape=(naction,))(y)
    elif advantage == 'naive':
      z = Lambda(lambda a: K.expand_dims(a[:,0], dim=-1) + a[:,1:], output_shape=(naction,))(y)
    else:
      assert False

    return x, z